In [1]:
import numpy as np

from QuICT.core.gate import *
from QuICT.algorithm.quantum_algorithm.random_walk import RandomWalk, Graph
from QuICT.simulation.state_vector import ConstantStateVectorSimulator

In [2]:
import matplotlib.pyplot as plt

def plot(data, size):
    data = [data[i] + data[i+1] for i in range(0, size*2, 2)]
    for i in range(size):
        plt.bar(i, data[i])

In [3]:
def classical_random_walk(size: int):
    position = [0] * size
    step = 0
    while step < size ** 3:
        related_prob = [0] * size
        for i in range(size):
            if position[i] != 0:
                add_idx = 0 if i == size - 1 else i + 1
                red_idx = size - 1 if i == 0 else i - 1
                related_prob[add_idx] += position[i]/2
                related_prob[red_idx] += position[i]/2

        if step == 0:
            position[0] = 1
        else:
            position = related_prob[:]

        break_point = True
        for prob in position:
            if not np.isclose(prob, 1/size):
                break_point = False
                break

        if break_point:
            break

        step += 1

    return position, step

In [4]:
print(classical_random_walk(11))

([0.0909081747562393, 0.09090996995131473, 0.09090832019226788, 0.09090969086161968, 0.09090871032544212, 0.09090922129123578, 0.09090922129123578, 0.09090871032544212, 0.09090969086161968, 0.09090832019226788, 0.09090996995131473], 295)


In [5]:
def circular_graph_generator(size):
    vertices = size
    edges = {}
    for i in range(size):
        addi = i + 1 if i != size - 1 else 0
        subi = i - 1 if i != 0 else size - 1
        edges[i] = [addi, subi]
    
    return Graph(vertices, edges)

In [3]:
# Build Circular Graph with different coin by vertices
vertices = 5
edges = {0: [4, 1], 1: [0, 2], 2: [1, 3], 3: [2, 4], 4: [3, 0]}
coin_operators = [H.matrix, H.matrix, U2.matrix, U1.matrix, H.matrix]
graph_circular = Graph(vertices, edges, coin_operators)

In [10]:
vertices = 11
graph_circular = circular_graph_generator(vertices)
# Quantum Walk Algorithm
rw = RandomWalk(
    step=200,
    graph=graph_circular,
    coin_operator=H.matrix
)

rw_cir = rw.circuit
print(rw_cir.size())

sim = ConstantStateVectorSimulator()
state = rw.run(sim, record_measured=False, shots = 100)

print(state)
# plot(state, vertices)

241800
[-1.22374917e-01-4.63292285e-12j -1.48503339e-02-7.35627858e-13j
  6.54094561e-02+2.50124920e-12j  1.85706900e-01+7.34579146e-12j
  1.38210651e-02+2.47626914e-13j  1.50326382e-01+6.50600627e-12j
  2.29013588e-01+9.67156793e-12j -3.38118537e-01-1.31914014e-11j
 -4.34758548e-02-1.88776599e-12j -7.69509546e-02-3.61823335e-12j
  2.86908782e-01+1.11685470e-11j  7.69509546e-02+2.81454894e-12j
  5.48076364e-01+2.21005040e-11j  3.38118537e-01+1.38019307e-11j
 -1.16851282e-01-5.23632973e-12j -1.50326382e-01-6.33813084e-12j
  3.81425226e-01+1.55716794e-11j -1.85706900e-01-7.70918906e-12j
 -1.21654983e-01-4.69042395e-12j  1.48503339e-02+8.49072474e-13j
 -1.20297444e-01-5.30206425e-12j -5.33450919e-14-1.68436637e-13j
  2.99695224e-13+1.79296137e-13j -3.87626382e-14-5.44867564e-14j
 -2.33716800e-13-2.52617858e-13j -3.02937183e-13-1.41872831e-13j
 -4.22651613e-14-8.47373127e-14j -1.56087797e-14-5.39503426e-15j
  2.61059961e-13-1.42571536e-13j  1.56708064e-13-9.26907437e-14j
  2.54250163e-14-1